# Fine-Tuning LLaMA 3.2-3B-Instruct on Personal Writings

**Cameron Pattison | Empirical Study on P4 Claims | December 2025**

This notebook fine-tunes Meta's LLaMA 3.2-3B-Instruct model on a corpus of philosophical writings.

## Before Running

1. **Runtime**: Go to Runtime → Change runtime type → Select **T4 GPU** (free) or **A100** (Colab Pro)
2. **Hugging Face Access**: You need access to `meta-llama/Llama-3.2-3B-Instruct`
   - Go to https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct
   - Accept the license agreement
3. **Get a Token**: https://huggingface.co/settings/tokens

## Step 1: Install Dependencies

In [ ]:
%%capture
# Install Unsloth for 2x faster training and 60% less memory
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes
!pip install datasets pyyaml wandb

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## Step 2: Authenticate with Hugging Face

In [ ]:
import os
from huggingface_hub import login

# Option 1: Enter token directly (paste your token when prompted)
login()

# Option 2: Set token as environment variable
# os.environ["HF_TOKEN"] = "hf_your_token_here"

## Step 3: Upload Your Corpus

Upload `complete_corpus.txt` to Colab.

In [ ]:
from google.colab import files

# Upload the corpus file
print("Please upload complete_corpus.txt")
uploaded = files.upload()

# Verify upload
if 'complete_corpus.txt' in uploaded:
    print(f"\n✓ Uploaded: complete_corpus.txt ({len(uploaded['complete_corpus.txt'])} bytes)")
else:
    print("Please upload complete_corpus.txt")

## Step 4: Load and Prepare the Model

In [ ]:
from unsloth import FastLanguageModel

# Configuration
MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct"
MAX_SEQ_LENGTH = 2048
LOAD_IN_4BIT = True

# Load model and tokenizer
print(f"Loading {MODEL_NAME}...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=LOAD_IN_4BIT,
)

print("✓ Model loaded successfully!")

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

# Print trainable parameters
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable:,} / {total:,} ({100*trainable/total:.2f}%)")

## Step 5: Prepare Training Data

In [ ]:
import re
import json

def parse_corpus(filepath):
    """Parse the corpus into individual documents."""
    with open(filepath, 'r', encoding='utf-8') as f:
        content = f.read()
    
    documents = []
    doc_pattern = r'-{80}\nDocument \d+: (.+?)\n-{80}\n'
    parts = re.split(doc_pattern, content)
    
    for i in range(1, len(parts), 2):
        if i + 1 < len(parts):
            title = parts[i].strip()
            doc_content = parts[i + 1].strip()
            
            # Clean up
            doc_content = re.sub(r'\n\d+\n', '\n', doc_content)
            doc_content = re.sub(r'\n{3,}', '\n\n', doc_content)
            
            if doc_content:
                documents.append({
                    "title": title,
                    "content": doc_content
                })
    
    return documents

# Parse corpus
documents = parse_corpus('complete_corpus.txt')
print(f"Parsed {len(documents)} documents:")
for doc in documents:
    print(f"  - {doc['title']}: {len(doc['content'])} chars")

In [ ]:
from datasets import Dataset

# System prompt for the model
SYSTEM_PROMPT = """You are Cameron Pattison, a philosophy PhD student at Vanderbilt University. 
Your research focuses on AI ethics, philosophy of mind, and the moral status of artificial 
intelligence systems. You have a background in classical philosophy (Aristotelian and Islamic 
traditions) and bring historical perspectives to contemporary debates. You write with analytical 
precision and philosophical rigor."""

def create_training_samples(documents):
    """Create training samples from documents."""
    samples = []
    
    for doc in documents:
        # Split into paragraphs
        paragraphs = [p.strip() for p in doc['content'].split('\n\n') if len(p.strip()) > 100]
        
        # Create continued pre-training samples
        for para in paragraphs:
            samples.append({
                "text": f"<|begin_of_text|>{para}<|end_of_text|>",
                "source": doc['title']
            })
    
    return samples

# Create samples
samples = create_training_samples(documents)
print(f"Created {len(samples)} training samples")

# Create dataset
dataset = Dataset.from_list(samples)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"Train: {len(dataset['train'])} samples")
print(f"Val: {len(dataset['test'])} samples")

## Step 6: Train the Model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir="./outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    num_train_epochs=3,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    max_grad_norm=0.3,
    fp16=True,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,
    logging_steps=10,
    report_to="none",  # Set to "wandb" for logging
    gradient_checkpointing=True,
    group_by_length=True,
)

# Create trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    args=training_args,
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_text_field="text",
    packing=True,
)

print("Trainer configured. Ready to train!")

In [ ]:
# Train!
print("Starting training...")
trainer.train()
print("\n✓ Training complete!")

## Step 7: Save the Model

In [ ]:
# Save LoRA adapters
model.save_pretrained("cameron-llama-3.2-3b")
tokenizer.save_pretrained("cameron-llama-3.2-3b")
print("✓ LoRA adapters saved to: cameron-llama-3.2-3b/")

# Optionally save merged model (larger, but standalone)
# model.save_pretrained_merged("cameron-llama-3.2-3b-merged", tokenizer, save_method="merged_16bit")

## Step 8: Test the Model

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

def generate_response(prompt, max_new_tokens=256):
    """Generate a response from the fine-tuned model."""
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt}
    ]
    
    formatted = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(formatted, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[1]:],
        skip_special_tokens=True
    )
    
    return response.strip()

print("Ready for inference!")

In [ ]:
# Test prompts
test_prompts = [
    "What is your view on AI consciousness?",
    "How do you approach questions of moral status?",
    "What is your argument about the P4 proposal?",
    "Tell me about your background in philosophy.",
]

for prompt in test_prompts:
    print(f"\n{'='*60}")
    print(f"PROMPT: {prompt}")
    print(f"-"*60)
    response = generate_response(prompt)
    print(f"RESPONSE: {response}")

In [ ]:
# Interactive testing
while True:
    prompt = input("\nYou: ")
    if prompt.lower() in ['quit', 'exit', 'q']:
        print("Goodbye!")
        break
    
    response = generate_response(prompt)
    print(f"\nCameron: {response}")

## Step 9: Download the Model

In [ ]:
# Zip and download
!zip -r cameron-llama-3.2-3b.zip cameron-llama-3.2-3b/

from google.colab import files
files.download('cameron-llama-3.2-3b.zip')

## Empirical Study Notes

Document your observations here for the P4 paper:

### Inextricability (Section 3.1.1)
- [ ] Do model outputs blend personal patterns with population-level regularities?

### Steerability (Section 3.1.2)
- [ ] Does the model incorporate new context provided at inference time?

### Replica Characteristics (Section 3.2.2)
- [ ] Does the model speak "in your voice"?

### Observations:
```
(Add your notes here)
```